In [2]:
import pandas as pd

In [37]:
df = pd.read_csv("Prediction-mgc.csv")

In [30]:
df["Acutal_answer"]=df["Acutal_answer"].str.replace("</code>","")

In [38]:
df

,Question,Acutal_answer,Predicted_answer
0,Problem:\nI have the following dataframe:\nind...,import pandas as pd\n\n\nindex = range(14)\nda...,import pandas as pd\nimport numpy as np\nindex...
1,Problem:\ni got an issue over ranking of date ...,import pandas as pd\n\n\ndf = pd.DataFrame({'I...,import pandas as pd\ndf = pd.DataFrame({'ID': ...
2,Problem:\nI have a DataFrame like :\n 0 ...,import pandas as pd\nimport numpy as np\n\ndf ...,import pandas as pd\ndf = pd.DataFrame({'0': [...
3,Problem:\nI have this Pandas dataframe (df):\n...,import pandas as pd\n\n\ndf = pd.DataFrame({'A...,import pandas as pd\ndf = pd.DataFrame({'A': [...
4,Problem:\nI have\n\ndf = pd.DataFrame.from_dic...,import pandas as pd\n\ndf = pd.DataFrame.from_...,import pandas as pd\ndf = pd.DataFrame.from_di...
...,...,...,...
98,Problem:\n\nI want to load a pre-trained word2...,import numpy as np\nimport pandas as pd\nimpor...,import numpy as np\nimport pandas as pd\nimpor...
99,Problem:\n\nI'm trying to slice a PyTorch tens...,import numpy as np\nimport pandas as pd\nimpor...,"import torch\nC = torch.LongTensor([[1, 3], [4..."
100,"Problem:\n\nGiven a 3d tenzor, say: batch x se...",import numpy as np\nimport pandas as pd\nimpor...,import numpy as np\nimport pandas as pd\nimpor...
101,Problem:\n\nI have the tensors:\n\nids: shape ...,import numpy as np\nimport pandas as pd\nimpor...,"import numpy as np\nids = np.array([[0,1,0],[1..."


In [39]:
import math
import numpy as np
from collections import Counter

def generate_ngram(text,n= 2,n_gram= False):
    '''
    N-Gram generator with parameters sentence
    n is for the number of n_grams
    The n_gram parameter removes repeating n_grams
    '''
    text = text.lower()  # converting to lower case
    str_arr = np.array(text.split())  # split to string arrays
    length = len(str_arr)

    word_list = []
    for i in range(length+1):
        if i < n:
            continue
        word_range = list(range(i-n,i))
        s_list = str_arr[word_range]
        string = ' '.join(s_list)  # converting list to strings
        word_list.append(string) # append to word_list
        if n_gram:
            word_list = list(set(word_list))
    return word_list


def bleu_score(reference,generated):
    '''
    Bleu score function is given the reference, or original text, and generated/machine-translated texts.
    '''
    gen_length = len(generated.split())
    ref_length = len(reference.split())

    # Brevity Penalty 
    if gen_length>ref_length:
        BP=1
    else:
        penality=1-(gen_length/ref_length)
        BP=np.exp(penality)

    # Clipped precision
    clipped_precision_score = []
    for i in range(1, 5):
        ref_n_gram = Counter(generate_ngram(reference,i))
        gen_n_gram = Counter(generate_ngram(generated,i))
        c = sum(gen_n_gram.values())
        for j in gen_n_gram:
            if j in ref_n_gram:
                if gen_n_gram[j] > ref_n_gram[j]:
                    gen_n_gram[j] = ref_n_gram[j]
            else:
                gen_n_gram[j] = 0

        clipped_precision_score.append(sum(gen_n_gram.values())/c)
    weights =[0.5]*2 #Modifying for bleu-2, would be [0.25]*4 for bleu-4
    s = (w_i * math.log(p_i) for w_i, p_i in zip(weights, clipped_precision_score))
    s = BP * math.exp(math.fsum(s))
    return s

In [40]:
bleuScoreSum = 0
for i in df.index:
    bleuScoreSum+=bleu_score(df["Acutal_answer"][i], df["Predicted_answer"][i])
print(bleuScoreSum/len(df))

0.601418116992347
